# O Elefante vidente

Notebook apresentado no [Floripa Tech Day 2022](https://www.floripatechday.com.br/o-evento/) com o intuito de ser um índice de informações sobre como podemos executar algoritmos de machine learning "dentro" do próprio Postgres.

## Versão: 0.9-beta

Esta versão beta foi utilziada para testar a hipótese se era possível treinar um modelo, armazená-lo e reusá-lo para previsão posterior, validando algumas referências. 
Como próximos passos os modelos serão melhorados e gráficos serão utilizados para compor a análise.

## O que veremos

* PostgreSQL
* Anaconda
* Jupyter
* VSCode
* KMeans

## Fontes: 

- https://www.postgresql.org/docs/14/plpython.html
- https://archive.ics.uci.edu/ml/index.php
- https://github.com/catherinedevlin/ipython-sql
- https://rpubs.com/HassanOUKHOUYA/K-means
- https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
- https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
- https://www.cybertec-postgresql.com/en/machine-learning-in-postgresql-part-1-kmeans-clustering/
- https://www.askpython.com/python/examples/plot-k-means-clusters-python

# Preparação do notebook

In [20]:
%load_ext sql
%sql postgresql://guedes@localhost:5435/guedes

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Limpeza inicial

In [50]:
%%sql
drop table if exists iris;
drop table if exists wine;
drop table if exists models;
drop table if exists red_wine;
drop table if exists white_wine;
drop function if exists show_tables();
drop function if exists varios_testes();
drop function if exists show_mnist();
drop function if exists show_mnist(out text);
drop function if exists show_mnist(out double precision, out double precision);
drop function if exists get_kmeans_centroids(text, text, integer);
drop function if exists kmeans(text, text[], integer);
drop function if exists multiout_simple_setof(integer, integer, integer);
drop function if exists multiout_simple_setof(integer);
drop function if exists predict_kmeans(text, text, integer, real[]);

 * postgresql://guedes@localhost:5435/guedes
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Criando funções auxiliares

### Função que testa imports e execução de consulta

In [51]:
%%sql
create or replace 
    function varios_testes()
    returns text
as $$
    import pandas as pd 
    import numpy as np
    import tensorflow as tf

    tables = plpy.execute('select schemaname, tablename, tableowner from pg_tables')

    columns = list(tables[0].keys())
    df = pd.DataFrame.from_records(tables, columns=columns)
    df['table'] = df['tablename'].str.upper()

    return 'tabelas: \n{}\n----\n TF Version: {}'.format(df, tf.__version__)
$$
language plpython3u;

 * postgresql://guedes@localhost:5435/guedes
Done.


[]

In [23]:
%%sql
select varios_testes();

 * postgresql://guedes@localhost:5435/guedes
1 rows affected.


varios_testes
tabelas: schemaname ... table0 pg_catalog ... PG_STATISTIC1 pg_catalog ... PG_TYPE2 pg_catalog ... PG_FOREIGN_TABLE3 pg_catalog ... PG_AUTHID4 pg_catalog ... PG_STATISTIC_EXT_DATA.. ... ... ...61 pg_catalog ... PG_LARGEOBJECT62 information_schema ... SQL_PARTS63 information_schema ... SQL_IMPLEMENTATION_INFO64 information_schema ... SQL_FEATURES65 information_schema ... SQL_SIZING[66 rows x 4 columns]---- TF Version: 2.4.1


#### Possível erro por conta da GLIBC

Se este erro ocorrer, uma atitude marota precisa ser tomada...

```
(psycopg2.errors.ExternalRoutineException) ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /home/guedes/anaconda3/envs/ml/lib/python3.9/site-packages/pandas/_libs/window/aggregations.cpython-39-x86_64-linux-gnu.so)
CONTEXT:  Traceback (most recent call last):
  PL/Python function "show_tables", line 2, in <module>
    import pandas as pd 
  PL/Python function "show_tables", line 47, in <module>
  PL/Python function "show_tables", line 47, in <module>
  PL/Python function "show_tables", line 0, in <module>
  PL/Python function "show_tables", line 69, in <module>
  PL/Python function "show_tables", line 156, in <module>
  PL/Python function "show_tables", line 151, in <module>
  PL/Python function "show_tables", line 0, in <module>
  PL/Python function "show_tables", line 11, in <module>
PL/Python function "show_tables"

[SQL: select * from show_tables();]
```

Forcei o link simbólico para a lib do meu env no Anaconda:

```bash
cd /usr/lib/x86_64-linux-gnu/
sudo rm libstdc++.so.6
sudo ln -s /home/guedes/anaconda3/envs/ml/lib/libstdc++.so.6.0.29
sudo mv libstdc++.so.6.29 libstdc++.so.6.28
```

... e reiniciei o PostgreSQL.

### Função usando Tensorflow

In [52]:
%%sql
create or replace 
    function show_mnist(
                out test_loss double precision, 
                out test_acc double precision
    )
    returns setof record
as $$
    import tensorflow as tf
    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5)
    test_loss, test_acc = model.evaluate(x_test, y_test)
    return [(test_loss, test_acc)]
$$
language plpython3u;

 * postgresql://guedes@localhost:5435/guedes
Done.


[]

In [53]:
%%sql
select * from show_mnist();

 * postgresql://guedes@localhost:5435/guedes
1 rows affected.


test_loss,test_acc
0.07613225281238556,0.9776999950408936


Não seria maravilhoso conseguir deixar esse modelo armazenado?

# Clusterizando dados

In [54]:
%%sql
create table iris(
    sepal_length real,
    sepal_width real,
    petal_length real,
    petal_width real,
    species varchar(20)
);

create table white_wine(
    fixed_acidity         real,
    volatile_acidity      real,
    citric_acid           real,
    residual_sugar        real,
    chlorides             real,
    free_sulfur_dioxide   real,
    total_sulfur_dioxide  real,
    density               real,
    pH                    real,
    sulphates             real,
    alcohol               real,
    quality               real
);


 * postgresql://guedes@localhost:5435/guedes
Done.
Done.


[]

Para armazenar os modelos gerados, vamos serializá-los na base de dados usando a tabela abaixo:

In [55]:
%%sql
create table models (
    model_name text primary key,
    model_dump bytea not null
);


 * postgresql://guedes@localhost:5435/guedes
Done.


[]

Podemos também fazer parte do código em nossa máquina em situações em que o banco pode não ter acesso a chamadas HTTPS externas. Abaixo um exemplo:

In [56]:
def load_data():
    import psycopg2
    import requests
    from io import BytesIO
    
    white_wine_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
    iris_data_url  = "https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"

    resp_white = BytesIO(requests.get(white_wine_url).content)
    resp_iris = BytesIO(requests.get(iris_data_url).content)

    conn = psycopg2.connect("host=localhost dbname=guedes user=guedes port=5435")
    cur = conn.cursor()

    with resp_white as f:
        next(f)
        cur.copy_from(f, 'white_wine', sep=';')

    with resp_iris as f:
        next(f)
        cur.copy_from(f, 'iris', sep=',')

    conn.commit()

E então executamos localmente...

In [57]:
load_data()

No Postgres seria algo mais ou menos como:


```sql
\copy iris from iris.data delimiter ','
\copy wine from winequality-white.csv csv header delimiter ';'
\copy wine from winequality-red.csv csv header delimiter ';'
```

In [58]:
white_wine_ = %sql select * from white_wine
iris_ = %sql select * from iris

 * postgresql://guedes@localhost:5435/guedes
4898 rows affected.
 * postgresql://guedes@localhost:5435/guedes
150 rows affected.


In [59]:
white_wine = white_wine_.DataFrame()
iris = iris_.DataFrame()

del(white_wine_)
del(iris_)

In [60]:
white_wine.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0


In [61]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [69]:
%%sql
create or replace 
    function kmeans(
                input_table text, 
                columns text[], 
                clus_num int
    )
returns bytea as
$$
    from pandas import DataFrame
    from sklearn.cluster import KMeans
    from pickle import dumps
    
    all_columns = ",".join(columns)
    if all_columns == "":
        all_columns = "*"
    
    rv = plpy.execute('SELECT %s FROM %s;' % (all_columns, plpy.quote_ident(input_table)))
    
    frame = []
    
    for i in rv:
        frame.append(i)
    df = DataFrame(frame)
    kmeans = KMeans(n_clusters=clus_num, random_state=0).fit(df._get_numeric_data())
    return dumps(kmeans)
$$
language plpython3u;

create or replace 
    function get_kmeans_centroids(
                    model_table text, 
                    model_column text, 
                    model_name text 
    )
returns real[] as
$$
    from pandas import DataFrame
    from pickle import loads
    
    rv = plpy.execute('SELECT %s FROM %s WHERE model_name = %s;' % (plpy.quote_ident(model_column), plpy.quote_ident(model_table), plpy.quote_literal(model_name)))
    model = loads(rv[0][model_column])
    ret = list(map(list, model.cluster_centers_))
    return ret
$$
language plpython3u;

create or replace 
    function predict_kmeans(
                model_table text, 
                model_column text, 
                model_name text, 
                input_values real[]
    )
returns int[] as
$$
    from pickle import loads
    
    rv = plpy.execute('SELECT %s FROM %s WHERE model_name = %s;' % (plpy.quote_ident(model_column), plpy.quote_ident(model_table), plpy.quote_literal(model_name)))
    
    model = loads(rv[0][model_column])
    ret = model.predict(input_values)
    return ret
$$
language plpython3u;

 * postgresql://guedes@localhost:5435/guedes
Done.
Done.
Done.


[]

In [74]:
%%sql
insert into models(model_name, model_dump) 
  select 'iris', 
         kmeans('iris', 
                array[]::text[], 
                3);

 * postgresql://guedes@localhost:5435/guedes
1 rows affected.


[]

In [78]:
%%sql
insert into models(model_name, model_dump) 
  select 'white_wine',
  kmeans('white_wine', 
                array[
                    'fixed_acidity', 
                    'volatile_acidity', 
                    'citric_acid', 
                    'residual_sugar', 
                    'chlorides', 
                    'free_sulfur_dioxide', 
                    'total_sulfur_dioxide', 
                    'density', 
                    'pH', 
                    'sulphates', 
                    'alcohol']::text[], 
                    10);

 * postgresql://guedes@localhost:5435/guedes
1 rows affected.


[]

In [75]:
%%sql 
select distinct 
       species, 
       predict_kmeans('models', 
                      'model_dump', 
                      'iris', 
                      array[[
                        petal_length, 
                        petal_width, 
                        sepal_length, 
                        sepal_width]]) 
    from iris;

 * postgresql://guedes@localhost:5435/guedes
5 rows affected.


species,predict_kmeans
setosa,[0]
versicolor,[0]
versicolor,[2]
virginica,[0]
virginica,[2]


In [79]:
%%sql
select distinct 
       quality, 
       predict_kmeans('models', 
                      'model_dump', 
                      'white_wine', 
                      array[[
                        fixed_acidity, 
                        volatile_acidity, 
                        citric_acid, 
                        residual_sugar, 
                        chlorides, 
                        free_sulfur_dioxide, 
                        total_sulfur_dioxide, 
                        density, 
                        pH, 
                        sulphates, 
                        alcohol]]) 
  from white_wine 
  order by 1;

 * postgresql://guedes@localhost:5435/guedes
59 rows affected.


quality,predict_kmeans
3.0,[3]
3.0,[6]
3.0,[8]
3.0,[7]
3.0,[4]
3.0,[2]
3.0,[9]
3.0,[1]
3.0,[0]
4.0,[6]


In [81]:
%%sql
select get_kmeans_centroids('models', 'model_dump', 'iris');

 * postgresql://guedes@localhost:5435/guedes
1 rows affected.


get_kmeans_centroids
"[[5.9016128, 2.748387, 4.3935485, 1.4338709], [5.006, 3.418, 1.464, 0.244], [6.85, 3.0736842, 5.7421055, 2.0710526]]"


In [82]:
%%sql
select get_kmeans_centroids('models', 'model_dump', 'white_wine');

 * postgresql://guedes@localhost:5435/guedes
1 rows affected.


get_kmeans_centroids
"[[7.0960503, 0.2944614, 0.3448833, 7.934829, 0.04933393, 30.989227, 162.24417, 0.99541, 3.182783, 0.49061042, 10.093118], [6.793891, 0.2750199, 0.31540504, 3.838247, 0.040480744, 22.570385, 93.05578, 0.99207354, 3.180478, 0.47573707, 11.2027445], [7.0259495, 0.3052057, 0.3664557, 10.537342, 0.05097785, 55.273735, 223.03798, 0.99702203, 3.163481, 0.51718354, 9.482279], [6.6880746, 0.25530434, 0.32159007, 5.135652, 0.041796274, 33.83851, 112.64534, 0.9927432, 3.200062, 0.48049688, 11.013975], [6.8208556, 0.2674198, 0.36850268, 9.183422, 0.052326202, 62.132355, 173.53476, 0.9958739, 3.1860428, 0.51657754, 9.822059], [6.716775, 0.25440842, 0.33612642, 7.1314425, 0.046358187, 47.663696, 143.26581, 0.99422836, 3.2063534, 0.48265803, 10.462399], [6.9714284, 0.29894456, 0.3428998, 8.513006, 0.051936034, 41.86034, 190.97868, 0.9959363, 3.1907036, 0.51, 9.78678], [7.357143, 0.365, 0.28142858, 5.0857143, 0.055714287, 123.85714, 335.14285, 0.99537283, 3.2757144, 0.5757143, 10.214286], [6.892759, 0.28117242, 0.31879312, 3.0606897, 0.039575864, 13.581035, 62.993103, 0.99180406, 3.1655173, 0.46448275, 11.1745405], [6.8832393, 0.29361972, 0.32695776, 5.325, 0.04518169, 23.79155, 130.43661, 0.99362445, 3.1907606, 0.4910141, 10.630747]]"
